## Movie Review Sentiment Analyisis

In [50]:
import pandas as pd 
import numpy as np 
import re 

In [51]:
%pwd

'/home/jakub/Python_Projects/NLP/Movie-Review-Project'

In [52]:
# open the recently created rotten tomatoes dataset for analysis
df = pd.read_csv('Rotten_Tomatoes_Dataset.csv')
df.head()

,Unnamed: 0,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content,threshold,...,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,sentiment
0,23,m/0814255,Kurt Loder,False,MTV,Rotten,1.0,2010-02-12,Sadly worthy of its dumping into the cinematic...,5,...,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76,Negative
1,44,m/0814255,Christopher Tookey,False,Daily Mail (UK),Rotten,1.0,2010-02-16,"Maybe if you're aged between eight and 12, or ...",5,...,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76,Negative
2,50,m/0814255,Tim Robey,True,Daily Telegraph (UK),Rotten,1.0,2010-02-16,A slab of market research in search of an actu...,5,...,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76,Negative
3,58,m/0814255,Anthony Quinn,True,Independent (UK),Rotten,1.0,2010-02-16,If they'd played it for laughs it might just h...,5,...,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76,Negative
4,61,m/0814255,Derek Malcolm,True,London Evening Standard,Rotten,1.0,2010-02-16,This ineffably wooden Chris Columbus adaptatio...,5,...,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76,Negative


In [53]:
df.columns

Index(['Unnamed: 0', 'rotten_tomatoes_link', 'critic_name', 'top_critic',
       'publisher_name', 'review_type', 'review_score', 'review_date',
       'review_content', 'threshold', 'movie_title', 'movie_info',
       'critics_consensus', 'content_rating', 'genres', 'directors', 'authors',
       'actors', 'original_release_date', 'streaming_release_date', 'runtime',
       'production_company', 'tomatometer_status', 'tomatometer_rating',
       'tomatometer_count', 'audience_status', 'audience_rating',
       'audience_count', 'tomatometer_top_critics_count',
       'tomatometer_fresh_critics_count', 'tomatometer_rotten_critics_count',
       'sentiment'],
      dtype='object')

In [54]:
df.describe()

,Unnamed: 0,review_score,runtime,tomatometer_rating,tomatometer_count,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
count,59037.000000,59037.000000,58674.000000,58969.000000,58969.000000,58926.000000,5.891000e+04,59037.000000,59037.000000,59037.000000
mean,212619.218643,3.737063,110.886594,68.910156,157.596771,68.852238,5.334537e+05,33.383709,118.964886,38.575791
std,125983.044495,1.859246,22.145725,31.308442,113.896312,21.240769,3.607354e+06,17.996908,111.227056,43.367737
min,23.000000,1.000000,5.000000,0.000000,5.000000,0.000000,5.000000e+00,0.000000,0.000000,0.000000
25%,102126.000000,1.000000,95.000000,43.000000,64.000000,53.000000,7.652000e+03,17.000000,29.000000,7.000000
50%,214224.000000,5.000000,107.000000,85.000000,135.000000,76.000000,3.641800e+04,36.000000,81.000000,22.000000
75%,321294.000000,5.000000,122.000000,93.000000,226.000000,86.000000,1.415010e+05,48.000000,183.000000,55.000000
max,431552.000000,5.000000,266.000000,100.000000,574.000000,100.000000,3.579764e+07,69.000000,497.000000,303.000000


In [55]:
df = df[['movie_title','genres','review_content','sentiment']] # select only the columns we want to work with 

In [56]:
df.head()

,movie_title,genres,review_content,sentiment
0,Percy Jackson & the Olympians: The Lightning T...,"Action & Adventure, Comedy, Drama, Science Fic...",Sadly worthy of its dumping into the cinematic...,Negative
1,Percy Jackson & the Olympians: The Lightning T...,"Action & Adventure, Comedy, Drama, Science Fic...","Maybe if you're aged between eight and 12, or ...",Negative
2,Percy Jackson & the Olympians: The Lightning T...,"Action & Adventure, Comedy, Drama, Science Fic...",A slab of market research in search of an actu...,Negative
3,Percy Jackson & the Olympians: The Lightning T...,"Action & Adventure, Comedy, Drama, Science Fic...",If they'd played it for laughs it might just h...,Negative
4,Percy Jackson & the Olympians: The Lightning T...,"Action & Adventure, Comedy, Drama, Science Fic...",This ineffably wooden Chris Columbus adaptatio...,Negative


## Data Preperation

In [57]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [58]:
# clean the review column 
def clean_text(text):
    """ 
    Change words to lower case, remove any punctuation and stop words 
    """
    text = text.lower()
    text = re.sub('[!|.|,|?|!|-|+]','',text)
    text = re.sub('\d','',text)
    text = ' '.join([word for word in text.split() if word not in (stopwords.words('english'))])
    return text 

In [59]:
clean_text('Hello 123?')

'hello'

In [60]:
cleaned = lambda x: clean_text(x)

In [61]:
df['review_content'] = df['review_content'].apply(cleaned)

In [62]:
df.head()

,movie_title,genres,review_content,sentiment
0,Percy Jackson & the Olympians: The Lightning T...,"Action & Adventure, Comedy, Drama, Science Fic...",sadly worthy dumping cinematic boneyard wintry...,Negative
1,Percy Jackson & the Olympians: The Lightning T...,"Action & Adventure, Comedy, Drama, Science Fic...",maybe aged eight exceptionally dim-witted may ...,Negative
2,Percy Jackson & the Olympians: The Lightning T...,"Action & Adventure, Comedy, Drama, Science Fic...",slab market research search actual movie,Negative
3,Percy Jackson & the Olympians: The Lightning T...,"Action & Adventure, Comedy, Drama, Science Fic...",they'd played laughs might worked fact earnest...,Negative
4,Percy Jackson & the Olympians: The Lightning T...,"Action & Adventure, Comedy, Drama, Science Fic...",ineffably wooden chris columbus adaptation aut...,Negative


## Model Building: Logistic Regression

In [88]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
#from sklearn.feature_extraction.text import TfidfVectorizer
#seperate from reviews and response

X = df['review_content']
y = df['sentiment']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [89]:
print(X_train.shape)
print(y_train.shape)

(47229,)
(47229,)


In [90]:
print(X_test.shape)
print(y_test.shape)

(11808,)
(11808,)


In [130]:
# train the logistic regression model
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
tvec = TfidfVectorizer()
clf = LogisticRegression()

In [131]:
model = Pipeline([('vectorizer',tvec),('classifier',clf)])
model.fit(X_train,y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier', LogisticRegression())])

In [132]:
pred = model.predict(X_test)
pred

array(['Positive', 'Negative', 'Negative', ..., 'Positive', 'Negative',
       'Positive'], dtype=object)

In [133]:
print(accuracy_score(y_test,pred))
print(confusion_matrix(y_test,pred))
# We get an accuracy score of 87 %

0.8788109756097561
[[2661 1128]
 [ 303 7716]]


In [134]:
#perform crossvalidation
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y, cv=10)

/home/jakub/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jakub/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

In [135]:
print(np.mean(scores)) # After performing a 10 k-fold cross validation we get an overall average score of 87% making it a consistent classifier

0.8750951269782898


In [136]:
#Try out a few random movie reviews to see how the model runs 
a = 'This is the best movie','This is a terrible move'
model.predict(list(a))

array(['Positive', 'Negative'], dtype=object)